<a href="https://colab.research.google.com/github/sowmyarshetty/NNClass/blob/main/AmazonHomeKitchenReviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install BERTopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [5]:
import pandas as pd
import dask.dataframe as dd
import gdown
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

In [6]:
# Mount Google Drive (For Colab Users)
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
metadatafileid = '1eo8HyJeC5_G9KCHItFsLdJWLcCPTsrS0'
reviewsfileid = '1LFUK3el3oLgjcl1WKy-MzalCHh8xnJk5'
metafilename = 'meta_Home_and_Kitchen_50k.jsonl'
reviewsfilename = 'Home_and_Kitchen_1M.jsonl'

metadataurl = f"https://drive.google.com/uc?export=download&id={metadatafileid}"
reviewsurl = f"https://drive.google.com/uc?export=download&id={reviewsfileid}"

gdown.download(metadataurl,metafilename, quiet=False)
gdown.download(reviewsurl, reviewsfilename, quiet=False)



Downloading...
From (original): https://drive.google.com/uc?export=download&id=1eo8HyJeC5_G9KCHItFsLdJWLcCPTsrS0
From (redirected): https://drive.google.com/uc?export=download&id=1eo8HyJeC5_G9KCHItFsLdJWLcCPTsrS0&confirm=t&uuid=a648b428-16e9-4660-9502-d69ad7d4dfe0
To: /content/meta_Home_and_Kitchen_50k.jsonl
100%|██████████| 185M/185M [00:01<00:00, 144MB/s]
Downloading...
From (original): https://drive.google.com/uc?export=download&id=1LFUK3el3oLgjcl1WKy-MzalCHh8xnJk5
From (redirected): https://drive.google.com/uc?export=download&id=1LFUK3el3oLgjcl1WKy-MzalCHh8xnJk5&confirm=t&uuid=bd73a497-0bf0-4815-b00e-f43e133b52b6
To: /content/Home_and_Kitchen_1M.jsonl
100%|██████████| 6.52G/6.52G [00:40<00:00, 160MB/s]


'Home_and_Kitchen_1M.jsonl'

* Read the json files into dataframes

In [8]:
df_meta = pd.read_json(metafilename,lines=True)
df_reviews = pd.read_json(reviewsfilename,lines=True)




*   Analyse the datasets




In [9]:
print(df_meta.columns)
print(df_reviews.columns)
pd.set_option("display.max_colwidth", None)
print(df_meta.head(1))

Index(['main_category', 'title', 'average_rating', 'rating_number', 'features',
       'description', 'price', 'images', 'videos', 'store', 'categories',
       'details', 'parent_asin', 'bought_together'],
      dtype='object')
Index(['rating', 'title', 'text', 'images', 'asin', 'parent_asin', 'user_id',
       'timestamp', 'helpful_vote', 'verified_purchase'],
      dtype='object')
  main_category  \
0   Amazon Home   

                                                                                                                                             title  \
0  Set of 4 Irish Coffee Glass Mugs Footed 10.5 oz.Thick Wall Glass For Coffee, tea, Cappuccinos, Mulled Ciders,Hot Chocolates, Ice cream and More   

   average_rating  rating_number  \
0             4.6             18   

                                                                                                                                                                                                        

* Extract the brand, color and price data from the details column of the metadata

In [10]:
df_meta_normalized = df_meta
df_metadetails = pd.json_normalize(df_meta_normalized["details"])
# df_meta_normalized = df_meta_normalized.drop(columns=["details"]).join(df_metadetails)  # Merge with main DataFrame
print(df_metadetails.columns)
df_meta_normalized = df_meta_normalized.drop(columns=["details","images","videos"]).join(df_metadetails[["Brand","Material","Color","Style","Pattern"]])
print(df_meta_normalized.columns)

Index(['Brand', 'Material', 'Color', 'Capacity', 'Style', 'Pattern',
       'Product Care Instructions', 'Unit Count', 'Product Dimensions',
       'Number of Items',
       ...
       'Best Sellers Rank.Carpet & Upholstery Cleaning Machines Accessories',
       'Best Sellers Rank.Sausage Stuffers',
       'Best Sellers Rank.Tiered Serving Trays & Platters',
       'Best Sellers Rank.Panel Track Blinds',
       'Best Sellers Rank.Kitchen Drains & Strainers',
       'Best Sellers Rank.Heating, Cooling & Air Quality',
       'Best Sellers Rank.Novelty Toys & Amusements',
       'Best Sellers Rank.Bottled Iced Tea',
       'Best Sellers Rank.Students Round Edge Scissors',
       'Best Sellers Rank.Honey Jars'],
      dtype='object', length=2224)
Index(['main_category', 'title', 'average_rating', 'rating_number', 'features',
       'description', 'price', 'store', 'categories', 'parent_asin',
       'bought_together', 'Brand', 'Material', 'Color', 'Style', 'Pattern'],
      dtype='object')

In [11]:
print(df_meta_normalized.head(1))

  main_category  \
0   Amazon Home   

                                                                                                                                             title  \
0  Set of 4 Irish Coffee Glass Mugs Footed 10.5 oz.Thick Wall Glass For Coffee, tea, Cappuccinos, Mulled Ciders,Hot Chocolates, Ice cream and More   

   average_rating  rating_number  \
0             4.6             18   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         features  \
0  [☕PERFECT IRISH COFFEE MUG: With ou

* Merge the data sets from the meta and reviews with the relevant columns with parent_asin as the join key and index

In [12]:
df_merged = df_reviews.merge(df_meta_normalized[['title','description','price','Brand','Material','Color','categories',"parent_asin"]],on='parent_asin',how='left')

* Look for reviews with missing product metadata

In [13]:
missing_asins_df = df_reviews[~df_reviews['parent_asin'].isin(df_meta_normalized['parent_asin'])]
print(missing_asins_df[['parent_asin']].drop_duplicates().count())
print(missing_asins_df[['parent_asin']].count())
print(len(df_reviews))
print(len(df_merged))
print(df_merged.columns)

parent_asin    1731834
dtype: int64
parent_asin    12359277
dtype: int64
13376345
13376345
Index(['rating', 'title_x', 'text', 'images', 'asin', 'parent_asin', 'user_id',
       'timestamp', 'helpful_vote', 'verified_purchase', 'title_y',
       'description', 'price', 'Brand', 'Material', 'Color', 'categories'],
      dtype='object')


* Remove the reviews which do not have product metadata

In [14]:
df_new = df_merged.dropna(subset = ['title_y','description','price','Brand','Material','Color','categories','parent_asin'])
print(len(df_new))

754079


In [15]:
df_new.to_csv('AmazonHomeKitchenReviews.csv')

In [16]:
 !cp /content/AmazonHomeKitchenReviews.csv /content/drive/MyDrive/